In [ ]:
import sys
sys.path.append('../')
sys.path.append('../../')

from bayesbridge.random.tilted_stable_dist.rand_exp_tilted_stable \
    import ExpTiltedStableDist
from random import normalvariate as norm_rv
from math import sqrt, gamma
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

## Gibbs sample a powered-exponential distribution using the tilted stable sampler.

In [ ]:
tilted_stable = ExpTiltedStableDist(seed=0)

def powered_exp_dist_gibbs(n_sample, n_burnin=0, exponent=.5, scale=1.):
    beta = 0.
    samples = np.zeros(n_sample + n_burnin)
    for i in range(n_burnin + n_sample):
        lshrink_sq = .5 / tilted_stable.rv(exponent / 2, (beta / scale) ** 2)
        lshrink = sqrt(lshrink_sq)
        beta = lshrink * norm_rv(0., 1.)
        samples[i] = beta
    samples = samples[n_burnin:]
    return samples

def powered_exp_pdf(x, exponent, scale=1., normed=True):
    pdf = np.exp(- np.abs(x / scale) ** exponent)
    if normed:
        pdf *= exponent / (2 * scale * gamma(exponent ** -1))
    return pdf

In [ ]:
n_sample = 10 ** 6
n_burnin = 10 ** 3
exponent = 1 / 2
samples = powered_exp_dist_gibbs(n_sample, n_burnin, exponent)

In [ ]:
plt.figure(figsize=(8, 5))
plt.rcParams['font.size'] = 20

plot_range = 25 * np.array([-1, 1])
bins = np.linspace(plot_range[0], plot_range[1], 100)
x = np.linspace(plot_range[0], plot_range[1], 10001)
plt.hist(samples, bins=bins, density=True)
plt.plot(x, powered_exp_pdf(x, exponent), 'tab:green')
plt.show()